In [1]:
# 네이버 검색 API 예제 - 블로그 검색
# 출처: https://developers.naver.com/docs/serviceapi/search/blog/blog.md#%EB%B8%94%EB%A1%9C%EA%B7%B8

import time
import os
import sys
import json
import re
import urllib.request
import openai
from utils import generate_text_davinci, NaverSearchAPI

client_id = ""
client_secret = ""
openai.api_key = "sk-"# **GPT  key 지정

naver_api = NaverSearchAPI(client_id=client_id, client_secret=client_secret)

In [3]:
query = "제주도 여름철 추천할만한 해수욕장"    
error:int=0
context:str=""
try:
    context, error = naver_api.search_naver(query=query)
    if error != 0:
        print(f'[search_naver] [error] error=>{error}')
except Exception as e:
    print(f'Exception=>{e}')
    error=1001
    
if error ==0:
    
    # text-davinci-003 모델에서, 프롬프트 길이가 총 1772 넘어가면 BadRequest('https://api.openai.com/v1/completions') 에러 남.
    # 따라서 context 길이가 1730 이상이면 1730까지만 처리함.
    if len(context) > 1730:
        context = context[0:1730]
    
    start_time = time.time()

    prompt = f'{context}\n\nQ:{query}?.2문장으로 답해줘\nA:'
    #print(prompt)    
    
    response = generate_text_davinci(gpt_model="text-davinci-003",
                                     prompt=prompt)

    # 소요된 시간을 계산합니다.
    end_time = time.time()
    elapsed_time = "{:.2f}".format(end_time - start_time)
    print(f'time:{elapsed_time}\n')

    print(f'답변:{response[0]}')

data:{'model': 'text-davinci-003', 'prompt': '정보의 바다    7월 제주도 가볼만한곳 10곳 추천 여행지\n7월 제주도 가볼만한곳 10곳 여행지 추천 오늘은 7월 제주도 가볼만한 여행지 10곳을 추천해볼까 합니다. 개인적으로는 두 번의 한달살기를 포함하여 꽤나 자주 제주도를 다녀왔었는데요, 그 제주 여행길에 가봤던 곳들 중에서 여름에 가볼만한 장소들로만 열 곳을 선정했습니다. 제주도는 워낙에나 볼거리들이 많아 열 곳만 선정하긴 쉽진 않지만, 더운 여름철에...\n\n여름 제주도 여행 가볼만한곳(ft.제주해수욕장) 여름피서여행  제주도가볼만한곳  제주도여행코스  제주해수욕장....\n여름철, 휴가철, 피서철 바캉스를 제주도 해수욕장에서 시원하게 즐겨봅시다. 제주도 여행 가볼만한, 제주도 해수욕장을 여러분들께 추천해드립니다. 아래, 제주도 해수욕장들 중에서  기볼만한 해수욕장들을 확인하시고, 즐거운 제주도 여름휴가여행을 계획 해 보세요. 아래, 제주도 가보고 싶은 해수욕장을 확인하시고, 눌러보세요!!   제주도 가볼고 싶은 해수욕장...\n\n7월 제주도 가볼만한곳 여름철 명소 Best 5\n오늘은 무더운 7월 제주도 가볼만한곳 중에서 여름철 가보면 좋은... 함덕해수욕장 제주도 7월 가볼만한곳 중에서 해수욕장을 빼놓을 수... 아이들과 해수욕을 즐기기에도 정말 좋은 곳이에요. 함덕 해수욕장의 7월... 제\n\n제주도 해수욕장 정보 총정리 15곳. [위치,장단점, 특징]\n일찍부터 해수욕을 즐길 수 있는 곳입니다. 여름이 아닌 다른 계절에... 여름철이 되면 피서객들로 야영지가 붐빌정도로 제주도 해수욕장... 금능해수욕장 근처 맛집 추천 BEST 10 제주 금능해수욕장 근처 숙소 BEST 7 4....\n\n휴가철 여름에 제주도에서 가볼만한곳 !\n동문시장 서귀포매일 올레시장 제주시 민속오일장 이상으로 제주도 여름에 가볼 만한 곳에 대한 추천을 마치겠습니다. 제주도는 자연과 문화가 조화로운 아름다운 장소로, 여름철 휴가를 계